In [ ]:
for item in cres:
    print("Number of missing value in ", item, ": ", cres[item].isna().sum())
    
#Extract the columns of the credits table
movie_id = cres['movie_id']
title = cres['title']
cast = cres['cast']
crew = cres['crew']

movie_id[0]

title[0]

# This line is to display the dictionary of cast
cast[0]

#This line is to display the dictionary of crew
crew[0]

num_cast = []
name_cast = []
for i in range(len(cast)):
    temp = eval(cast[i]) #Convert to list of real dictionaries
    num_cast.append(len(temp)) # Append the number of cast to the list
    temp_name = []
    count = 0
    for ca in temp:
        if count<3:
            count = count+1
            temp_name.append(str.lower(ca['name'].replace(" ","")))
    name_cast.append(temp_name)
print(num_cast[0]) #Test for number of cast in the first film
print("Length of num_cast column", len(num_cast))
print("Length of name_cast column", len(name_cast))
print("List of 3 cast names in the first movie:", name_cast[0])

num_crew = []
director = []
for i in range(len(crew)):
    temp = eval(crew[i]) #Convert the sting-based dictionaries to list of real dictionaries
    num_crew.append(len(temp)) # Append the number of cast to the list
    for cr in temp:
        if cr['job'] == 'Director':
            director.append(str.lower(cr['name'].replace(" ","")))
            
print(director[1])
print("Length of num_crew column:", len(num_crew))
print("Director of the first movie:", director[0])

#### Summary: <br>
From the ***credits*** dataset, we have 6 variales: <br>
***- movie_id***<br>
***- title***<br>
***- num_cast***: Used for regression model<br>
***- num_crew***: Used for regression model<br>
***- name_cast***: Used for recommendation system<br>
***- director***: Used for recommendation system<br>
with totally 4803 record.
****

### Extract the data from movies dataset

In [ ]:
for item in moviesdata:
    missval = moviesdata[item].isna().sum()
    print("Number of missing value in ", item, ": ", missval)

Here, we observe some missing values in variables homepage, release_date, overview, runtime, tagline. <br>
We will then fill these missing values with some default ones.

In [ ]:
moviesdata['homepage'] = moviesdata['homepage'].fillna('')
moviesdata['overview'] = moviesdata['overview'].fillna('')
moviesdata['release_date'] = moviesdata['release_date'].fillna('')
moviesdata['runtime'] = moviesdata['runtime'].fillna(0)
moviesdata['tagline'] = moviesdata['tagline'].fillna('')

#### Extract the columns of the table

In [ ]:
budget = moviesdata['budget']
genres = moviesdata['genres']
# homepage: We ignore this variable
# id: We have the id from the credit dataset
keywords = moviesdata['keywords']
original_language = moviesdata['original_language']
original_title = moviesdata['original_title']
popularity = moviesdata['popularity']
production_companies = moviesdata['production_companies']
production_countries = moviesdata['production_countries']
release_day = moviesdata['release_date']
revenue = moviesdata['revenue']
runtime = moviesdata['runtime']
spoken_languages = moviesdata['spoken_languages']
status = moviesdata['status']
tagline = moviesdata['tagline']
# title: already from the credit dataset
vote_average = moviesdata['vote_average']
vote_count = moviesdata['vote_count']

#### Count the number of genres and the list of genres for each movies

In this part, we extract the number of genres, as a possible features for predicting revenue. Also, we extract 3 first genres, lower case them and remove space for recommendation later.

In [ ]:
genres[0] #explore first element in genres

num_genres = []
name_genres = []
for i in range(len(genres)):
    temp = eval(genres[i])
    num_genres.append(len(temp))
    temp_name = []
    count = 0
    for ge in temp:
        if count<4:
            count = count+1
            temp_name.append(str.lower(ge['name'].replace(" ","")))
    name_genres.append(temp_name)
print(num_genres[0])
print(name_genres[0])

In [ ]:
genres_list = set()
for na in name_genres:
    for n in na:
        genres_list.add(n)
genres_list = list(genres_list)
genres_list

#### Transform this categorical variable in to one-hot encoding

In [ ]:
genres_d = pd.DataFrame()
for genres in genres_list:
    temp = []
    for i in range(len(name_genres)):
        temp.append(1 if genres in name_genres[i] else 0)
    genres_d[genres] = pd.Series(temp).values
    moviesdata[genres] = pd.Series(temp).values
    
genres_d.head()
moviesdata.head()
moviesdata[genres]

#### Extract the keywords

In [ ]:
num_keywords = []
name_keywords = []
for i in range(len(keywords)):
    temp = eval(keywords[i])
    num_keywords.append(len(temp))
    temp_name = []
    count = 0
    for key in temp:
        if count<3:
            count = count+1
            temp_name.append(str.lower(key['name'].replace(" ","")))
    name_keywords.append(temp_name)
print(num_keywords[0])
print(name_keywords[0])

#### Convert all features to dataframe

In [ ]:
num_cast_d = pd.DataFrame(num_cast)
num_crew_d = pd.DataFrame(num_crew)
budget_d = pd.DataFrame(budget)
num_genres_d = pd.DataFrame(num_genres)
num_keywords_d = pd.DataFrame(num_keywords)
popularity_d = pd.DataFrame(popularity)
runtime_d = pd.DataFrame(runtime)
vote_average_d = pd.DataFrame(vote_average)
vote_count_d = pd.DataFrame(vote_count)
revenue_d = pd.DataFrame(revenue)

### 3.2. Normalize the distribution of the varibales by using log scale and remove outliers

In [ ]:
data_o = pd.concat([revenue_d, budget_d, num_cast_d, num_crew_d, genres_d, popularity_d, vote_count_d, vote_average_d], axis = 1).reindex(budget_d.index)
data_o.columns = ['revenue', 'budget', 'num_cast', 'num_crew', 'sciencefiction','action','western','comedy','mystery','war','documentary','history','adventure','romance','crime','family','tvmovie','drama','music','animation','fantasy','horror','foreign','thriller', 'popularity', 'vote_count', 'vote_average']
data_o.shape

from sklearn import preprocessing
data_o1 = data_o.copy() #copy of data_o
data_o1.loc[:, ['revenue', 'budget', 'popularity', 'vote_count', 'vote_average']] = preprocessing.normalize(data_o1.loc[:, ['revenue', 'budget', 'popularity', 'vote_count', 'vote_average']])
data_o1.loc[:, ['revenue', 'budget', 'popularity', 'vote_count', 'vote_average']]

### Measure the knewness and kurtosis of data 

In [ ]:
data_o.loc[:, ['revenue', 'budget', 'popularity', 'vote_count', 'vote_average']].skew()
data_o.loc[:, ['revenue', 'budget', 'popularity', 'vote_count', 'vote_average']].kurt()
data_o.loc[:, ['revenue', 'budget', 'popularity', 'vote_count', 'vote_average']]

### Plot the graphs for each of the variables

In [ ]:
f, axes = plt.subplots(5, 3, figsize=(16, 12))

count = 0
for var in data_o.loc[:, ['revenue', 'budget', 'popularity', 'vote_count', 'vote_average']]:
    sb.boxplot(data_o[var], orient = "h", ax = axes[count,0])
    sb.histplot(data_o[var], ax = axes[count,1])
    sb.violinplot(data_o[var], orient = "h", ax = axes[count,2])
    count += 1

We can see that the distribution of this data is positve skew. Therefore, we proceed to some normalization and cut the outlier to make it more uniform.

In [ ]:
data_process = data_o
data_process.shape

We remove all records with '0' value, as when we transform to log scale, and log(0) will go to -infinity. 

In [ ]:
data_process = data_process[data_process['budget'] > 0]
data_process = data_process[data_process['revenue'] > 0]
data_process = data_process[data_process['vote_count'] > 0]
data_process = data_process[data_process['popularity'] > 0]
data_process.shape

#### Transform the dataset into log scale to make it more uniform

In [ ]:
budget_log = pd.DataFrame(np.log(data_process['budget']))
revenue_log = pd.DataFrame(np.log(data_process['revenue']))
popularity_log = pd.DataFrame(np.log(data_process['popularity']))
vote_count_log = pd.DataFrame(np.log(data_process['vote_count']))
data_log = pd.concat([budget_log, revenue_log, num_cast_d, num_crew_d, genres_d, popularity_log, vote_count_log, vote_average_d], axis = 1).reindex(budget_log.index)
data_log.columns = ['budget', 'revenue', 'num_cast', 'num_crew', 'sciencefiction','action','western','comedy','mystery','war','documentary','history','adventure','romance','crime','family','tvmovie','drama','music','animation','fantasy','horror','foreign','thriller', 'popularity', 'vote_count', 'vote_average']
data_log.shape

Here, we use a function called ***zscore*** from stats in scipy. The function of this function is to to measure the distance of the point to mean over the standard deviation. If more than 3 times or the point lies out of 3 standard deviation, we cut that record. 

In [ ]:
data_log = data_log[(np.abs(stats.zscore(data_log['budget'])) < 3)]
data_log = data_log[(np.abs(stats.zscore(data_log['revenue'])) < 3)]
data_log = data_log[(np.abs(stats.zscore(data_log['popularity'])) < 3)]
data_log = data_log[(np.abs(stats.zscore(data_log['vote_count'])) < 3)]
data_log.shape

f, axes = plt.subplots(5, 3, figsize=(18, 12))

count = 0
for var in data_o.loc[:, ['revenue', 'budget', 'popularity', 'vote_count', 'vote_average']]:
    sb.boxplot(data_log[var], orient = "h", ax = axes[count,0])
    sb.histplot(data_log[var], ax = axes[count,1])
    sb.violinplot(data_log[var], orient = "h", ax = axes[count,2])
    count += 1
    
data_log.loc[:, ['revenue', 'budget', 'popularity', 'vote_count', 'vote_average']].skew()
data_log.loc[:, ['revenue', 'budget', 'popularity', 'vote_count', 'vote_average']].kurt()

Now, our data is more uniform and less skewed.